In [1]:
import pandas as pd
import numpy as np
from scipy.stats import shapiro, ttest_ind, mannwhitneyu

## Actions 

- Load the dataset
- Data Cleaning: Check missing rows and duplicates
- create the patient groups 
- Create the new yellow and orange columns
Perform the tests

### Questions
- PostOp:CompConditionCodes did not recieve codes
- RiskScores:Medication is this pre-operative medicine?
- PathwayDetails:CeIndication what is the code for this and what do the numbers represent. 
- Duplicate rows: 


## 1. Load the Dataset

In [2]:
#  opnen excel file and convert to dataframe

data = pd.read_csv('Carotid_data.csv')
data.head()

C:\Users\bkhalidson.CMC\AppData\Local\Temp\ipykernel_16352\3099976692.py:3: DtypeWarning: Columns (22,26) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('Carotid_data.csv')


,Patient:AgeAtSurgery,Patient:GenderCode,PathwayDetails:CeIndication,Symptom_Month/Year,Indications:IndicationSideCode,Indications:CeStenosisIpsilCode,Indications:CeStenosisContraCode,Indications:PrevIpsilInd,Indications:PrevContraInd,RiskScores:Comorbidities,...,PostOp:CompConditionCodes,PostOp:StrokeTypeCode,PostOp:CompCnInjuryInd,NvrEpisode:DischargeStatusCode,LengthOfStay,FollowUp:Readmission30DaysInd,FollowUp:CeFuClinicStrokeInd,FollowUp:DiedWithin30Days,DaysDischargeToFollowUp,FollowUp:FuClinicApptDate
0,83.2,2,4,2/2021,2,2.0,5.0,False,False,3,...,0,NaN,False,1,60,False,False,False,NaN,NaN
1,75.3,2,4,6/2018,1,2.0,0.0,False,False,2|4,...,0,NaN,False,1,24,False,False,False,NaN,NaN
2,70.5,1,4,1/2018,2,4.0,2.0,True,False,1|2|4,...,0,NaN,False,1,57,True,False,False,42.0,26/04/2018
3,75.6,1,4,11/2017,2,2.0,NaN,False,NaN,1|2|4|6,...,2,NaN,False,1,20,False,False,NaN,NaN,NaN
4,70.8,1,4,3/2017,2,5.0,5.0,False,NaN,2|3|4,...,0,NaN,False,0,8,NaN,NaN,NaN,NaN,NaN


## 2.1 Data Cleaning: Missing Data 

In [3]:
# Calculate the number of missing values for each column
missing_values = data.isnull().sum()

# Calculate the percentage of missing values for each column
missing_percentage = (missing_values / len(data)) * 100

# Create a report DataFrame
missing_report = pd.DataFrame({
    'Number of Missing Values': missing_values,
    'Percentage of Missing Values (%)': missing_percentage
})

# Sort the report by the number of missing values in descending order
missing_report = missing_report.sort_values(by='Number of Missing Values', ascending=False)

missing_report.to_csv('missing_data_report.csv')

In [59]:
missing_report

,Number of Missing Values,Percentage of Missing Values (%)
PostOp:StrokeTypeCode,19105,99.838002
RiskScores:PeriopMedCode,16494,86.193562
RiskScores:PatientFrailty,10021,52.367266
FollowUp:DiedWithin30Days,7064,36.914716
Indications:PrevContraInd,7000,36.580268
Indications:CeStenosisContraCode,6902,36.068144
DaysDischargeToFollowUp,6199,32.394440
FollowUp:FuClinicApptDate,6199,32.394440
PostOp:CriticalCareStayDays,3553,18.567099
DaysFromSymptomToSurgery,1201,6.276129


## 2.2 Duplicates

In [4]:
# check if dataframe has dupicates
duplicates = data.duplicated().sum()
duplicate_rows = data[data.duplicated()]
duplicate_rows

# drop duplicates
data = data.drop_duplicates()


## 3. Feature Engineering

### 3.1 Patient Groups 

In [5]:
# Create 'Patient Group' column based on "RiskScores:PreoperativeRankinScore"
data['Patient Group'] = data['RiskScores:PreoperativeRankinScore'].apply(lambda x: 1 if x in [0, 1, 2] else (2 if x in [3, 4, 5] else None))

data[['RiskScores:PreoperativeRankinScore', 'Patient Group']].head()


,RiskScores:PreoperativeRankinScore,Patient Group
0,5,2
1,5,2
2,5,2
3,5,2
4,5,2


### 3.2 Yellow Highlights

This includes but are not limited to: 
- RiskScores:Medication
- RiskScores:PeriopMedCode
- Anaesthesia:AnaestheticTypeCodes
- PostOp:CompConditionCodes
- RiskScores:Comorbidities

In [6]:
def create_columns_based_on_multiple_scores(data, column_name):
    # Get unique scores in the column after stripping whitespaces
    unique_scores = set()
    for value in data[column_name].dropna():
        scores = map(str.strip, str(value).split('|'))
        unique_scores.update(scores)
    
    # Create new columns based on these unique scores
    for score in unique_scores:
        new_column = f"{column_name}_{score}"
        data[new_column] = data[column_name].apply(lambda x: True if score in map(str.strip, str(x).split('|')) else False)
    
    return data, unique_scores

In [7]:
yellow_highlights = ['RiskScores:Medication', 'RiskScores:PeriopMedCode', 'Anaesthesia:AnaestheticTypeCodes', 'PostOp:CompConditionCodes', 'RiskScores:Comorbidities']
unique_scores_dict = {}

for col in yellow_highlights: 
    data, unique_scores = create_columns_based_on_multiple_scores(data, col)
    unique_scores_dict[col] = unique_scores

unique_scores_dict

{'RiskScores:Medication': {'0', '1', '2', '3', '4', '5', '6', '7', '8', '9'},
 'RiskScores:PeriopMedCode': {'1', '2', '3'},
 'Anaesthesia:AnaestheticTypeCodes': {'1',
  '11',
  '12',
  '2',
  '3',
  '4',
  '5',
  '6',
  '7',
  '8',
  '9'},
 'PostOp:CompConditionCodes': {'0',
  '1',
  '13',
  '15',
  '17',
  '18',
  '2',
  '3',
  '4',
  '5'},
 'RiskScores:Comorbidities': {'0',
  '1',
  '2',
  '3',
  '4',
  '5',
  '6',
  '7',
  '8',
  '9'}}

In [8]:
data.columns

Index(['Patient:AgeAtSurgery', 'Patient:GenderCode',
       'PathwayDetails:CeIndication', 'Symptom_Month/Year',
       'Indications:IndicationSideCode', 'Indications:CeStenosisIpsilCode',
       'Indications:CeStenosisContraCode', 'Indications:PrevIpsilInd',
       'Indications:PrevContraInd', 'RiskScores:Comorbidities',
       'RiskScores:SmokingStatus', 'RiskScores:Creatinine', 'RiskScores:ASA',
       'RiskScores:AtrialFibrillation', 'RiskScores:PreoperativeRankinScore',
       'RiskScores:Medication', 'RiskScores:PeriopMedCode',
       'RiskScores:ThrombolysisPreoperative', 'RiskScores:PatientFrailty',
       'DaysFromSymptomToSurgery', 'Anaesthesia:AnaestheticTypeCodes',
       'CarotidEndar:IpsilTypeCode', 'CarotidEndar:IpsilShuntInd',
       'PostOp:ProcDestCode', 'PostOp:CriticalCareStayDays',
       'PostOp:CompReturnTheatreInd', 'PostOp:CompCcReadmissionInd',
       'PostOp:CompConditionCodes', 'PostOp:StrokeTypeCode',
       'PostOp:CompCnInjuryInd', 'NvrEpisode:DischargeSt

### 3.2 Orange Highlights 

The include but are not limited to: 
- PathwayDetails:CeIndication
- Indications:IndicationSideCode
- Indications:CeStenosisIpsilCode
- Indications:CeStenosisContraCode
- RiskScores:SmokingStatus
- CarotidEndar:IpsilTypeCode
- PostOp:ProcDestCode




In [50]:
# Columns to be one-hot encoded
columns_to_encode = [
    "Indications:IndicationSideCode", 
    "PathwayDetails:CeIndication", 
    "Indications:CeStenosisIpsilCode", 
    "Indications:CeStenosisContraCode",
    "RiskScores:SmokingStatus", 
    "CarotidEndar:IpsilTypeCode", 
    "PostOp:ProcDestCode"
]

# One-hot encoding the specified columns
encoded_data = pd.get_dummies(data, columns=columns_to_encode, prefix=columns_to_encode)

### 3.3 Pre process (all together)

In [15]:
## Data pre-process function

def create_columns_based_on_multiple_scores(data, column_name):
    # Get unique scores in the column after stripping whitespaces
    unique_scores = set()
    for value in data[column_name].dropna():
        scores = map(str.strip, str(value).split('|'))
        unique_scores.update(scores)
    
    # Create new columns based on these unique scores
    for score in unique_scores:
        new_column = f"{column_name}_{score}"
        data[new_column] = data[column_name].apply(lambda x: True if score in map(str.strip, str(x).split('|')) else False)
    
    return data, unique_scores


def generate_yellow_highlighted_columns(data):
    yellow_highlights = ['RiskScores:Medication', 'RiskScores:PeriopMedCode', 'Anaesthesia:AnaestheticTypeCodes', 'PostOp:CompConditionCodes', 'RiskScores:Comorbidities']
    unique_scores_dict = {}

    for col in yellow_highlights: 
        data, unique_scores = create_columns_based_on_multiple_scores(data, col)
        unique_scores_dict[col] = unique_scores

    return data, unique_scores_dict

def generate_orange_highlighted_columns(data):
    columns_to_encode = [
        "Indications:IndicationSideCode", 
        "Patient:GenderCode",
        "PathwayDetails:CeIndication", 
        "Indications:CeStenosisIpsilCode", 
        "Indications:CeStenosisContraCode",
        "PostOp:StrokeTypeCode",
        "RiskScores:SmokingStatus", 
        "CarotidEndar:IpsilTypeCode", 
        "PostOp:ProcDestCode"
    ]

    encoded_data = pd.get_dummies(data, columns=columns_to_encode, prefix=columns_to_encode)

    return encoded_data

def pre_process(data):
    # Drop the duplicates
    data = data.drop_duplicates()

    # Create 'Patient Group' column based on "RiskScores:PreoperativeRankinScore"
    data['Patient Group'] = data['RiskScores:PreoperativeRankinScore'].apply(lambda x: 1 if x in [0, 1, 2] else (2 if x in [3, 4, 5] else None))

    # Create new columns based on multiple scores - yellow highlighted columns
    data, unique_scores_dict = generate_yellow_highlighted_columns(data)
    
    # One hot code the orange highlighted columns
    data = generate_orange_highlighted_columns(data)

    return data, unique_scores_dict

In [55]:
carotid_data = pd.read_csv('Carotid_data.csv')
carotid_data, unique_scores_dict = pre_process(carotid_data)
carotid_data.to_csv('carotid_data_preprocessed.csv')

C:\Users\bkhalidson.CMC\AppData\Local\Temp\ipykernel_16352\3344183124.py:1: DtypeWarning: Columns (22,26) have mixed types. Specify dtype option on import or set low_memory=False.
  carotid_data = pd.read_csv('Carotid_data.csv')
C:\Users\bkhalidson.CMC\AppData\Local\Temp\ipykernel_16352\4018724083.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Patient Group'] = data['RiskScores:PreoperativeRankinScore'].apply(lambda x: 1 if x in [0, 1, 2] else (2 if x in [3, 4, 5] else None))
C:\Users\bkhalidson.CMC\AppData\Local\Temp\ipykernel_16352\4018724083.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

### 3.3 Create the tests 

There are different tests we will have to do based on the data we will be handling. Such differences in data are: 
- Continous Data
- Binary 
- Categorical 

#### 3.3.1 Continuous data

The tests that use continuous data are: 
- A: Patient:AgeAtSurgery
- L: RiskScores:Creatinine
- M: RiskScores:ASA
- S: RiskScores:PatientFrailty
- T: DaysFromSymptomToSurgery
- Y: PostOp:CriticalCareStayDays
- AF: LengthOfStay
- AJ: DaysDischargeToFollowUp

In [52]:
continuous_data = {
    'A': {'col':'Patient:AgeAtSurgery'},
    'L': {'col':'RiskScores:Creatinine'},
    'M': {'col':'RiskScores:ASA'},
    'S': {'col':'RiskScores:PatientFrailty'},
    'T': {'col':'DaysFromSymptomToSurgery'},
    'Y': {'col':'PostOp:CriticalCareStayDays'},
    'AF': {'col':'LengthOfStay'},
    'AJ': {'col':'DaysDischargeToFollowUp'},
}

def test_continuous_data(data, column_name, group_column='Patient Group', alpha=0.05):
    """
    Test a continuous data column for differences between two groups.
    
    Parameters:
    - data: DataFrame containing the data.
    - column_name: Name of the column with continuous data to test.
    - group_column: Name of the column with group labels (default 'Patient Group').
    - alpha: Significance level for the test (default 0.05).
    
    Returns:
    - test_used: The name of the statistical test used.
    - p_value: The p-value from the statistical test.
    - significance: Whether there's a significant difference or not.
    - avg_group1: Average of group 1.
    - avg_group2: Average of group 2.
    - higher_avg_group: Indicates which group has a higher average.
    - count_group1: Number of patients in group 1.
    - count_group2: Number of patients in group 2.
    """
    # Separate the data by group
    group1_data = data[data[group_column] == 1][column_name].dropna()
    group2_data = data[data[group_column] == 2][column_name].dropna()

    # Count the number of patients in each group
    count_group1 = len(group1_data)
    count_group2 = len(group2_data)

    # Calculate the average for each group
    avg_group1 = group1_data.mean()
    avg_group2 = group2_data.mean()

    # Determine which group has a higher average
    if avg_group1 > avg_group2:
        higher_avg_group = "Group 1"
    elif avg_group1 < avg_group2:
        higher_avg_group = "Group 2"
    else:
        higher_avg_group = "Both groups have the same average"

    # Check for normality in both groups
    _, p1 = shapiro(group1_data)
    _, p2 = shapiro(group2_data)

    # If data is normally distributed in both groups, use T-test
    if p1 > 0.05 and p2 > 0.05:
        _, p_value = ttest_ind(group1_data, group2_data)
        test_used = "T-test"
    # Otherwise, use Mann-Whitney U test
    else:
        _, p_value = mannwhitneyu(group1_data, group2_data)
        test_used = "Mann-Whitney U test"

    # Determine significance
    if p_value < alpha:
        significance = "Significant difference"
    else:
        significance = "No significant difference"

    return test_used, p_value, significance, avg_group1, avg_group2, higher_avg_group, count_group1, count_group2


for test in continuous_data:
    test_used, p_value, significance, avg_group1, avg_group2, higher_avg_group, count_group1, count_group2 = test_continuous_data(carotid_data, continuous_data[test]['col'])
    continuous_data[test]['test_used'] = test_used
    continuous_data[test]['p_value'] = p_value
    continuous_data[test]['significance'] = significance
    continuous_data[test]['avg_group1'] = avg_group1
    continuous_data[test]['avg_group2'] = avg_group2
    continuous_data[test]['higher_avg_group'] = higher_avg_group
    continuous_data[test]['group1_size'] = count_group1
    continuous_data[test]['group2_size'] = count_group2
    

# Convert continuous_data to dataframe
continuous_data_df = pd.DataFrame(continuous_data).T
continuous_data_df.to_csv('continuous_data_tests.csv')



c:\Users\bkhalidson.CMC\Documents\coding_projects\envs\lucinda\Lib\site-packages\scipy\stats\_morestats.py:1882: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


Next steps

- know which patient group is higher 
- Add the average of the patient groups 
- Add the number of patients in each group for each test
- remove missing patients in isolation 
- chi-squared add the frequencies of True and false
- Send over the results and send over the instructions on how to run the program. 


#### 3.3.2 Binary Data

- H: (Previous ipsilateral surgery, exact name not provided)  - No column
- I: (Previous contralateral surgery, exact name not provided)
- N: ( AF, exact name not provided)
- R: (Perioperative thrombolysis, exact name not provided)
- W: (use of Intraoperative shunt, exact name not provided)
- AD: (to Post-operative nerve injury, exact name not provided)
- AG: (30-day readmission after discharge, exact name not provided)
- AH: (30-day post-discharge stroke, exact name not provided)
- AI: (30-day post-op death, exact name not provided)
- Z: Return to theatre post operatively (Yes/No)
- AA: Return to critical care post discharge (Yes/No)

In [48]:
from scipy.stats import chi2_contingency, fisher_exact

def test_binary_data(data, column_name, group_column='Patient Group', alpha=0.05):
    """
    Test a binary data column for differences between two groups using either a Chi-Squared test or Fischer's Exact test.
    
    Parameters:
    - data: DataFrame containing the data.
    - column_name: Name of the column with binary data to test.
    - group_column: Name of the column with group labels (default 'Patient Group').
    - alpha: Significance level for the test (default 0.05).
    
    Returns:
    - test_used: The name of the statistical test used.
    - test_statistic: The test statistic value.
    - significance: Whether there's a significant difference or not.
    - positives_group1: Number of True values in group 1.
    - positives_group2: Number of True values in group 2.
    - higher_avg_group: Indicates which group has more True values.
    - count_group1: Number of patients in group 1.
    - count_group2: Number of patients in group 2.
    """
    # Separate the data by group
    group1_data = data[data[group_column] == 1][column_name].dropna()
    group2_data = data[data[group_column] == 2][column_name].dropna()

    # Count the number of True values for each group
    positives_group1 = group1_data.sum()
    positives_group2 = group2_data.sum()

    # Determine which group has more True values
    if positives_group1 > positives_group2:
        higher_positives_group = "Group 1"
    elif positives_group1 < positives_group2:
        higher_positives_group = "Group 2"
    else:
        higher_positives_group = "Both groups have the same number of True values"

    # Count the number of patients in each group
    count_group1 = len(group1_data)
    count_group2 = len(group2_data)

    # Create a contingency table
    contingency_table = pd.crosstab(data[group_column], data[column_name])
    
    # If any cell in the table has an expected count less than 5, use Fischer's Exact test
    chi2, p_value, _, expected = chi2_contingency(contingency_table)
    if contingency_table.shape == (2, 2) and (expected < 5).any():
        _, p_value = fisher_exact(contingency_table)
        test_used = "Fischer's Exact Test"
        test_statistic = None  # Fisher's exact test doesn't have a typical test statistic like chi2
    elif contingency_table.shape != (2, 2):
        test_used = "Not applicable - Table shape not 2x2"
        test_statistic = None
        p_value = None
    else:
        test_used = "Chi-Squared Test"
        test_statistic = chi2

    # Determine significance
    if p_value and p_value < alpha:
        significance = "Significant difference"
    elif p_value:
        significance = "No significant difference"
    else:
        significance = "Cannot determine due to table shape"

    return test_used, p_value, significance, positives_group1, positives_group2, higher_positives_group, count_group1, count_group2




In [49]:

binary_data_columns = pd.read_csv('Columns_to_questions.csv')
binary_data_columns = binary_data_columns[binary_data_columns['Data Type'] == 'Binary']
binary_data_columns[['Column', 'Name of Column']]

# binary data columns is a dataframe with column letters and names of columns. Create a dictionary with column letters as keys and a dictionary with key as 'col' and  list of the names that contain the name of the column in carotid_data's columsn as values
binary_data_columns_dict = {}
for col in binary_data_columns['Column']:
    binary_data_columns_dict[col] = list(carotid_data.columns[carotid_data.columns.str.contains(binary_data_columns[binary_data_columns['Column'] == col]['Name of Column'].values[0])])

# binary_data_columns_dict is a dictionary with column letters as keys and a list of names as columns as values. Create a dictionary with column letters as keys and a dictionary with key as 'col' and value as the name of the column in the same list as values. If the list has more than one element, increment the key name by 1 i.e A_1, A_2, etc.
binary_data_columns_dict_2 = {}
for col in binary_data_columns_dict:
    if len(binary_data_columns_dict[col]) == 1:
        binary_data_columns_dict_2[col] = {'col': binary_data_columns_dict[col][0]}
    else:
        for i in range(len(binary_data_columns_dict[col])):
            binary_data_columns_dict_2[f"{col}_{i+1}"] = {'col': binary_data_columns_dict[col][i]}

binary_data_columns_dict_2

C:\Users\bkhalidson.CMC\AppData\Local\Temp\ipykernel_16352\15824486.py:1: DtypeWarning: Columns (0,1,2,3,4,23,27) have mixed types. Specify dtype option on import or set low_memory=False.
  binary_data_columns = pd.read_csv('Columns_to_questions.csv')


{'B_1': {'col': 'Patient:GenderCode_1'},
 'B_2': {'col': 'Patient:GenderCode_2'},
 'C_1': {'col': 'PathwayDetails:CeIndication_1'},
 'C_2': {'col': 'PathwayDetails:CeIndication_2'},
 'C_3': {'col': 'PathwayDetails:CeIndication_3'},
 'C_4': {'col': 'PathwayDetails:CeIndication_4'},
 'C_5': {'col': 'PathwayDetails:CeIndication_5'},
 'E_1': {'col': 'Indications:IndicationSideCode_1'},
 'E_2': {'col': 'Indications:IndicationSideCode_2'},
 'E_3': {'col': 'Indications:IndicationSideCode_3'},
 'F_1': {'col': 'Indications:CeStenosisIpsilCode_1.0'},
 'F_2': {'col': 'Indications:CeStenosisIpsilCode_2.0'},
 'F_3': {'col': 'Indications:CeStenosisIpsilCode_3.0'},
 'F_4': {'col': 'Indications:CeStenosisIpsilCode_4.0'},
 'F_5': {'col': 'Indications:CeStenosisIpsilCode_5.0'},
 'G_1': {'col': 'Indications:CeStenosisContraCode_0.0'},
 'G_2': {'col': 'Indications:CeStenosisContraCode_1.0'},
 'G_3': {'col': 'Indications:CeStenosisContraCode_2.0'},
 'G_4': {'col': 'Indications:CeStenosisContraCode_3.0'},
 

In [53]:
for test in binary_data_columns_dict_2:
    test_used, p_value, significance, positives_group1, positives_group2, higher_positives_group, count_group1, count_group2 = test_binary_data(carotid_data, binary_data_columns_dict_2[test]['col'])
    binary_data_columns_dict_2[test]['test_used'] = test_used
    binary_data_columns_dict_2[test]['p_value'] = p_value
    binary_data_columns_dict_2[test]['significance'] = significance
    binary_data_columns_dict_2[test]['TRUE_count_group1'] = positives_group1
    binary_data_columns_dict_2[test]['TRUE_count_group2'] = positives_group2
    binary_data_columns_dict_2[test]['higher_positives_group'] = higher_positives_group
    binary_data_columns_dict_2[test]['group1_size'] = count_group1
    binary_data_columns_dict_2[test]['group2_size'] = count_group2

binary_data_df = pd.DataFrame(binary_data_columns_dict_2).T
binary_data_df.to_csv('binary_data_tests.csv')
# test_used, p_value, significance, positives_group1, positives_group2, higher_positives_group, count_group1, count_group2 = test_binary_data(carotid_data, 'Patient:GenderCode_1')
# test_used, p_value, significance, positives_group1, positives_group2, higher_positives_group, count_group1, count_group2

#### 3.3.3 Categorical Data

C - is there a significant difference in patients having surgery for reason 1,2,3,4 or 5 between the two groups (ie 5 different statistical tests) Not sure

In [24]:
def test_categorical_data(data, column_name, group_column='Patient Group', alpha=0.05):
    """
    Test a categorical data column for differences between two groups using a Chi-Squared test.
    
    Parameters:
    - data: DataFrame containing the data.
    - column_name: Name of the column with categorical data to test.
    - group_column: Name of the column with group labels (default 'Patient Group').
    - alpha: Significance level for the test (default 0.05).
    
    Returns:
    - test_used: The name of the statistical test used.
    - test_statistic: The test statistic value.
    - significance: Whether there's a significant difference or not.
    """
    # Create a contingency table
    contingency_table = pd.crosstab(data[group_column], data[column_name])
    
    # Use Chi-Squared test
    chi2, p_value, _, _ = chi2_contingency(contingency_table)
    test_used = "Chi-Squared Test"
    test_statistic = chi2

    # Determine significance
    if p_value < alpha:
        significance = "Significant difference"
    else:
        significance = "No significant difference"

    return test_used, test_statistic, significance
